# Importing Packages

In [1]:
import pandas as pd
import fasttext_class
from sklearn.model_selection import train_test_split
from pathlib import Path
import fasttext
import os

RAW_DATA_PATH = 'raw_data/'
GENERATED_DATA_PATH = 'Gen_files/'
MERGED_DATA_PATH ='Merged/'
SHUFFLED_MERGED_DATA_PATH = 'Shuffled_merged/'
DATA_FINAL_PATH = 'data_final/'


## Generate the Files

In [2]:
def read_all_texts(path_to_folders, outfile_path):
    init_model=fasttext_class.fasttext_c()
    for folder in os.listdir(f'{path_to_folders}'):
        for filename in os.listdir(f'{path_to_folders}/{folder}'):
            with open(f'{path_to_folders}/{folder}/{filename}', encoding='utf-8') as f:
                init_model.gen_training(path= f'{path_to_folders}/{folder}', label= f'{folder}', outfile_path=f'{outfile_path}/{folder}')


In [3]:
read_all_texts(RAW_DATA_PATH, GENERATED_DATA_PATH)

[ERROR] No training data.


# Merging Files

In [4]:
def merge_file(path_to_folder, outfile_path):
    for file in os.listdir(f'{path_to_folder}'):
            with open(f'{path_to_folder}/{file}', encoding='utf-8') as fp: 
                data = fp.read() 
                data += '\n'
                file_lst=[]
                for filename in os.listdir(path_to_folder):
                    with open(f'{path_to_folder}/{filename}', encoding='utf-8') as f:
                        file_content = pd.read_csv(f, sep='\t', header=None)
                        file_lst.append(file_content)
                        
                merged = pd.concat(file_lst)
                merged.to_csv(f'{outfile_path}/' + 'merged', header=False, index=False, sep='\t')

In [5]:
merge_file(GENERATED_DATA_PATH, MERGED_DATA_PATH)

## Shuffling the Data

In [6]:
def shuffler(path_to_folder, outfile_path):
    for file in os.listdir(f'{path_to_folder}'):
            with open(f'{path_to_folder}/{file}', encoding='utf-8') as fp:
                df=pd.read_csv(fp, sep='\t', header=None)
                df = df.sample(frac=1).reset_index(drop=True)
    return df
    df.to_csv(f'{outfile_path}/' + 'shuffled_merged' + '.txt', header=False, index=False, sep='\t')

In [7]:
shuffled_merged=shuffler(MERGED_DATA_PATH, SHUFFLED_MERGED_DATA_PATH)

## Creating train and test columns

In [8]:
X_train, X_test , y_train, y_test = train_test_split(shuffled_merged[1], shuffled_merged[0], test_size=0.33, random_state=42)

## Creating train file

In [9]:
train=pd.DataFrame(X_train)

In [10]:
train.insert(0, '', y_train, True)

In [11]:
#saving train file
train.to_csv(DATA_FINAL_PATH + 'train_data.txt', sep='\t', header=None, index=False)

## Creating test file

In [12]:
test=pd.DataFrame(X_test)

In [13]:
test.insert(0, '', y_test, True)

In [14]:
#saving test file
test.to_csv(DATA_FINAL_PATH + 'test_data.txt', sep='\t', header=None, index=False)

## Loading the files- Train

In [15]:
with open(DATA_FINAL_PATH + 'train_data.txt', encoding='utf-8') as train_txt:
    final_train_data=train_txt.read()


## Loading the files- Test

In [16]:
with open(DATA_FINAL_PATH + 'test_data.txt', encoding='utf-8') as test_txt:
    final_test_data=test_txt.read()

## Creating the model

In [17]:
model_1=fasttext_class.fasttext_c(train_data=str(DATA_FINAL_PATH + 'train_data.txt'))

In [18]:
model_1.fasttext_test(DATA_FINAL_PATH + 'test_data.txt')

(100, 0.96, 0.96)